## Spark Scheduler.

El Spark Scheduler es el responsable de planificar y distribuir las tareas en un clúster de Spark. Su objetivo es optimizar la ejecución de las operaciones definidas en un job (trabajo) de Spark, asegurando que los recursos del clúster se utilicen de manera eficiente.

### Funciones principales del Spark Scheduler:

#### 1. **División de Jobs en Stages**:

- Un job en Spark se divide en stages (etapas) basándose en las dependencias entre las operaciones (transformaciones wide y narrow).

- Las operaciones narrow (como map o filter) no requieren un shuffle y se ejecutan en un solo stage.

- Las operaciones wide (como groupBy o join) requieren un shuffle y dividen el job en múltiples stages.

#### 2. **Planificación de Tareas (Tasks)**:

- Cada stage se divide en tasks (tareas), que son las unidades más pequeñas de trabajo.

- Cada task se ejecuta en un executor (nodo de trabajo) en el clúster.

#### 3. **Asignación de Recursos**:

- El scheduler asigna recursos (CPU, memoria) a las tareas, asegurándose de que los executors tengan los recursos necesarios para ejecutar las tareas de manera eficiente.

#### 4. **Gestión de Dependencias**:

- El scheduler maneja las dependencias entre stages y tareas, asegurándose de que las tareas se ejecuten en el orden correcto.

#### 5. **Tolerancia a Fallos**:

- Si una tarea falla, el scheduler la vuelve a programar en otro executor para garantizar que el job se complete.

### Componentes del Spark Scheduler
**a) DAGScheduler**:

- Divide el job en stages basándose en las dependencias entre las operaciones.

- Crea un DAG (Directed Acyclic Graph) que representa el flujo de ejecución del job.

- Envía los stages al TaskScheduler.

**b) TaskScheduler**:

- Se encarga de asignar tareas a los executors disponibles en el clúster.

- Comunica con el Cluster Manager (como YARN, Mesos o Kubernetes) para obtener recursos.

**c) Cluster Manager**:

- Gestiona los recursos del clúster (nodos de trabajo, memoria, CPU).

- Asigna executors a las aplicaciones de Spark.

### Proceso de Ejecución en Spark
**1. El usuario define un job** (por ejemplo, una consulta SQL o una operación de RDD).

**2. El DAGScheduler** divide el job en stages y crea un DAG.

**3. El TaskScheduler** asigna tareas a los executors.

**4. Los executors** ejecutan las tareas y devuelven los resultados al driver.

**5. El proceso se repite** hasta que todas las tareas del job se completan.

### : Flujo del Spark Scheduler

In [ ]:
Job (Usuario)
   |
   v
DAGScheduler (Divide en Stages)
   |
   v
TaskScheduler (Asigna Tasks a Executors)
   |
   v
Executors (Ejecutan Tasks)
   |
   v
Resultados (Devueltos al Driver)

![Plataforma Apache Spark](plataforma-spark.png)

### ¿Cómo funcioan Spark Scheduler?

**1. Driver y Executors**:

- Cuando ejecutas una aplicación Spark, se inician dos tipos de procesos:

  - **Driver**: Es el proceso principal que ejecuta el código de la aplicación (por ejemplo, tu programa en Scala, Python o Java). Aquí es donde vive el Spark Scheduler.

  - **Executors**: Son procesos que se ejecutan en los nodos del cluster y se encargan de ejecutar las tareas asignadas por el Driver.

**2. Spark Scheduler en el Driver**:

- El Spark Scheduler (que incluye el DAGScheduler y el TaskScheduler) es parte del Driver.

- Su trabajo es planificar y coordinar la ejecución de tareas en los executors del cluster.

- El Driver se comunica con el Cluster Manager (como YARN, Mesos o Kubernetes) para obtener recursos (executors) y luego asigna tareas a esos executors.

**3. Cluster Manager**:

- El Cluster Manager es el responsable de gestionar los recursos del cluster (nodos, memoria, CPU).

- Cuando creas un cluster, estás configurando el Cluster Manager (por ejemplo, YARN o Kubernetes), pero no estás creando un Spark Scheduler directamente.

- El Spark Scheduler es parte de la aplicación Spark que se ejecuta en el Driver, no del cluster en sí.

### ¿Qué pasa cuando creamos un cluster?
Cuando creas un cluster (por ejemplo, con YARN, Kubernetes o Spark Standalone), estás configurando un entorno donde:

**1. El Cluster Manager** gestiona los recursos (nodos, memoria, CPU).

**2.** Cuando envías una aplicación Spark al cluster:

- El Driver de la aplicación se inicia (ya sea en el cluster o en tu máquina local, dependiendo de la configuración).

- El Spark Scheduler (dentro del Driver) se comunica con el Cluster Manager para obtener executors.

- Los executors ejecutan las tareas asignadas por el Spark Scheduler.

### ¿Cuándo se crea una SparkSession?
Cuando ejecutas una aplicación Spark, debes crear explícitamente una SparkSession al inicio del programa. 

**ejemplo**:

In [ ]:
import org.apache.spark.sql.SparkSession

// Crear una SparkSession
val spark = SparkSession.builder()
  .appName("Mi Aplicación Spark") // Nombre de la aplicación
  .master("local[*]")             // Ejecutar en local con todos los núcleos
  .getOrCreate()                  // Crear o reutilizar una SparkSession existente

**Explicación del código:**  


- **appName("MiPrimeraAplicacionSpark")**: Asigna un nombre a la aplicación, que aparecerá en la interfaz web de Spark.

- **master("local[*]")**: Especifica que la aplicación se ejecutará en modo local, utilizando todos los núcleos disponibles.

- **.getOrCreate()**: Crea una nueva SparkSession o reutiliza una existente.


| Entorno   | Configuración en `.master()`                          |
|-----------|------------------------------------------------------|
| Local     | `"local[*]"` (usa todos los cores disponibles)       |
| Standalone | `"spark://<host>:<puerto>"` (ejemplo: `"spark://192.168.1.100:7077"`) |
| YARN      | `"yarn"` (modo clúster en Hadoop)                    |
| Mesos     | `"mesos://<host>:<puerto>"`                          |
| Kubernetes | `"k8s://<kubernetes-master>:<puerto>"`              |

- **Local**: Usa todos los núcleos disponibles en la máquina local.

- **Standalone**: Conecta a un clúster de Spark en modo independiente.

- **YARN**: Ejecuta Spark en un clúster de Hadoop usando YARN como gestor de recursos.

- **Mesos**: Ejecuta Spark en un clúster gestionado por Apache Mesos.

- **Kubernetes**: Ejecuta Spark en un clúster de Kubernetes.

#### Conclusión: Si corres en un clúster, reemplaza "local[*]" con la URL de tu clúster o el modo adecuado (yarn, mesos, etc.)

### Opciones para .master() según el entorno

### ¿Dónde se ejecuta la SparkSession?
**La SparkSession** se ejecuta en el **Driver**, que es el proceso principal de tu aplicación Spark. El Driver es responsable de:

**1.** Crear y gestionar la SparkSession.

**2.** Planificar tareas (a través del Spark Scheduler).

**3.** Enviar tareas a los executors en el cluster.

**4.** Recopilar resultados.

#### Relación entre SparkSession, SparkContext y Cluster Manager
- **SparkSession**:

  - Es el punto de entrada de alto nivel para usar Spark.

  - Gestiona el SparkContext internamente.

- **SparkContext**:

  - Es el objeto de bajo nivel que representa la conexión con el cluster.

  - Gestiona la comunicación con el Cluster Manager.

- **Cluster Manager**:

  - Es el responsable de asignar recursos (executors) al SparkContext.

#### Flujo de ejecución:

**1. El SparkContext** se comunica con el clúster para crear un driver y asignar executors a los nodos del clúster.

**2. Cuando ejecutas una acción** como rdd.collect() o se envía una aplicación Spark, el SparkContext envía un DAG de la aplicación al Spark Scheduler.

**3. El Spark Scheduler divide el trabajo en stages y tasks** según las dependencias (narrow y wide).

**4: El Spark Scheduler distribuye las tareas** a los executors disponibles en el clúster.
    
**5. Los executors** ejecutan las tareas asignadas y devuelven los resultados al driver a través del SparkContext.

#### Ejemplo de Flujo
**1.** Creas una SparkSession en tu aplicación.

**2.** La SparkSession crea un SparkContext.

**3.** El SparkContext se comunica con el Cluster Manager para obtener recursos.

**4.** El Spark Scheduler (dentro del Driver) planifica tareas y las envía a los executors.

Los executors ejecutan las tareas y devuelven los resultados al Driver.

#### ¿Qué es el Core de Spark?
El core de Spark es el módulo base de Apache Spark y proporciona las funcionalidades esenciales para la ejecución distribuida, como:

**1.** Gestión de memoria.

**2.** Planificación de tareas (Spark Scheduler).

**3.** Distribución de datos (RDDs, particiones).

**4.** Comunicación entre nodos.

**5.** Tolerancia a fallos.

El Spark Scheduler es uno de los componentes más importantes del core, ya que se encarga de coordinar la ejecución de tareas en el clúster.



![Arquitectura Apache Spark](arquitectura-spark.png)

#### **El Shell de Spark en Scala**

- La forma más sencilla de interactuar con Spark es a través del shell de Scala, que se inicia con el comando:

In [ ]:
spark-shell

- Este shell inicia una SparkSession automáticamente, permitiendo ejecutar código Scala directamente en un entorno interactivo.